In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Required Libraries


In [2]:
import string
import re
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity



# Importing necessary files

In [3]:
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movies_metadata.csv')
credits = pd.read_csv('/content/drive/My Drive/Colab Notebooks/credits.csv')
keywords = pd.read_csv('/content/drive/My Drive/Colab Notebooks/keywords.csv')
links_small = pd.read_csv('/content/drive/My Drive/Colab Notebooks/links_small.csv')


<ipython-input-3-a3db4cfc5a7d>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movies_metadata.csv')


# Preprocessing

In [4]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [6]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [8]:
movies.drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1, inplace = True)

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 45466 non-null  object 
 1   budget                45466 non-null  object 
 2   genres                45466 non-null  object 
 3   id                    45466 non-null  object 
 4   original_language     45455 non-null  object 
 5   original_title        45466 non-null  object 
 6   overview              44512 non-null  object 
 7   popularity            45461 non-null  object 
 8   production_companies  45463 non-null  object 
 9   production_countries  45463 non-null  object 
 10  release_date          45379 non-null  object 
 11  revenue               45460 non-null  float64
 12  runtime               45203 non-null  float64
 13  spoken_languages      45460 non-null  object 
 14  tagline               20412 non-null  object 
 15  vote_average       

In [10]:
#rows with inconsistent data types
movies.drop([19730, 29503, 35587], inplace=True)

In [11]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [12]:
movies['id'] = movies['id'].astype('int64')
movies = movies.merge(keywords, on='id').merge(credits, on='id')
movies.head()

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [13]:
#filtering movies
movies = movies[movies['id'].isin(links_small)]
movies.shape

(9219, 20)

In [14]:
movies['original_language'] = movies['original_language'].fillna('')
movies['runtime'] = movies['runtime'].fillna(0)
movies['tagline'] = movies['tagline'].fillna('')

movies.dropna(inplace=True)

<ipython-input-14-a832e02d9f53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['original_language'] = movies['original_language'].fillna('')
<ipython-input-14-a832e02d9f53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['runtime'] = movies['runtime'].fillna(0)
<ipython-input-14-a832e02d9f53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [15]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9207 entries, 0 to 41669
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 9207 non-null   object 
 1   budget                9207 non-null   object 
 2   genres                9207 non-null   object 
 3   id                    9207 non-null   int64  
 4   original_language     9207 non-null   object 
 5   original_title        9207 non-null   object 
 6   overview              9207 non-null   object 
 7   popularity            9207 non-null   object 
 8   production_companies  9207 non-null   object 
 9   production_countries  9207 non-null   object 
 10  release_date          9207 non-null   object 
 11  revenue               9207 non-null   float64
 12  runtime               9207 non-null   float64
 13  spoken_languages      9207 non-null   object 
 14  tagline               9207 non-null   object 
 15  vote_average          920

In [16]:
def convert(obj):
  x = []
  for i in literal_eval(obj):
    x.append(i['name'])
  return x

#Simple Recommender System using Cosine Similarity


In [17]:
new_df = movies[['id', 'original_title', 'overview', 'genres', 'cast', 'keywords']]

In [18]:
new_df['genres'] = new_df['genres'].apply(convert)
new_df['keywords'] = new_df['keywords'].apply(convert)
new_df['cast'] = new_df['cast'].apply(convert)

<ipython-input-18-0f8a3bf6e607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['genres'] = new_df['genres'].apply(convert)
<ipython-input-18-0f8a3bf6e607>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['keywords'] = new_df['keywords'].apply(convert)
<ipython-input-18-0f8a3bf6e607>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [19]:
new_df.head()

,id,original_title,overview,genres,cast,keywords
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o..."
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug..."


In [20]:
new_df['overview'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [21]:
new_df['overview'] = new_df['overview'].apply(lambda x: x.split())

<ipython-input-21-f086b9df7e4b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['overview'] = new_df['overview'].apply(lambda x: x.split())


In [22]:
new_df.head()

,id,original_title,overview,genres,cast,keywords
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Animation, Comedy, Family]","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Adventure, Fantasy, Family]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Romance, Comedy]","[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o..."
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Comedy, Drama, Romance]","[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug..."


In [23]:
new_df['genres'] = new_df['genres'].apply(lambda x: [i.replace(' ', '') for i in x])

<ipython-input-23-5016e4fb745d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['genres'] = new_df['genres'].apply(lambda x: [i.replace(' ', '') for i in x])


In [24]:
new_df['keywords'] = new_df['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
new_df['cast'] = new_df['cast'].apply(lambda x: [i.replace(' ', '') for i in x])

<ipython-input-24-2573bc33a430>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['keywords'] = new_df['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
<ipython-input-24-2573bc33a430>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['cast'] = new_df['cast'].apply(lambda x: [i.replace(' ', '') for i in x])


In [25]:
new_df['tags'] = new_df['overview'] + new_df['genres'] + new_df['cast'] + new_df['keywords']

<ipython-input-25-8039c47fe434>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['overview'] + new_df['genres'] + new_df['cast'] + new_df['keywords']


In [26]:
new_df2 = new_df[['id', 'original_title', 'tags']]

In [27]:
new_df2.head()

,id,original_title,tags
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,..."
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a..."
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ..."
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ..."
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr..."


In [28]:
new_df2['tags'] = new_df2['tags'].apply(lambda x: ' '.join(x))

<ipython-input-28-348b12792fe8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['tags'] = new_df2['tags'].apply(lambda x: ' '.join(x))


In [29]:
new_df2.head()

,id,original_title,tags
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [30]:
new_df2['tags'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Animation Comedy Family TomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife"

In [31]:
new_df2['tags'] = new_df2['tags'].apply(lambda x: x.lower())
new_df2.head()

<ipython-input-31-5c87c8b0a10d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['tags'] = new_df2['tags'].apply(lambda x: x.lower())


,id,original_title,tags
0,862,Toy Story,"led by woody, andy's toys live happily in his ..."
1,8844,Jumanji,when siblings judy and peter discover an encha...
2,15602,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,just when george banks has recovered from his ...


In [32]:
#convert text documents into numerical representations
tv = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = tv.fit_transform(new_df2['tags']).toarray()


In [33]:
len(tv.get_feature_names_out())

5000

In [34]:
pts = PorterStemmer()

In [35]:
def stem(text):
  l = []
  for i in text.split():
    l.append(pts.stem(i))
    return ' '.join(l)

In [36]:
new_df2['tags'] = new_df2['tags'].apply(stem)

<ipython-input-36-8ac23776d2a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['tags'] = new_df2['tags'].apply(stem)


In [37]:
cosine_similarity(vectors).shape

(9207, 9207)

In [38]:
similarity = cosine_similarity(vectors)

In [39]:
new_df2.reset_index(inplace=True)

In [40]:
def recommend_movies(title):
  try:
    index = new_df2[new_df2['original_title']==title].index[0]
    distance = similarity[index]
    movies = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies:
      print(new_df2.iloc[i[0]].original_title)
  except:
    print('Movie not found')


In [41]:
recommend_movies('Toy Story')

Toy Story 2
Toy Story 3
The 40 Year Old Virgin
Toy Story of Terror!
Condorman


In [42]:
import pickle

In [43]:
pickle.dump(new_df2, open('/content/drive/My Drive/Colab Notebooks/movies.pkl', 'wb'))
pickle.dump(similarity, open('//content/drive/My Drive/Colab Notebooks/similarity.pkl', 'wb'))

In [44]:
import os
